In [ ]:
import PyKDL

# create a vector
v = PyKDL.Vector(1,3,5)

# create a rotation from Roll Pitch, Yaw angles
r1 = PyKDL.Rotation.RPY(1.2, 3.4, 0)

# create a rotation from XYZ Euler angles
r2 = PyKDL.Rotation.EulerZYX(0, 1, 0)

# create a rotation from a rotation matrix
r3 = PyKDL.Rotation(1,0,0, 0,1,0, 0,0,1)

# create a frame from a vector and a rotation
f = PyKDL.Frame(r2, v)

In [3]:
import numpy as np
import time, sys, os
import h5py
import logging
import scipy.linalg
import scipy.linalg as LA
from scipy.optimize import minimize
import matplotlib

import os
from os.path import join, expanduser
%matplotlib inline
import matplotlib.pyplot as plt
##LOGGER = logging.getLogger(__name__)

#### gather all the cartesian points and velocities
+ data is thus arranged:
  - data = [
             \zeta^0, \zeta^1, \zeta^\star; 
             \dot{\zeta^0}, \dot{\zeta^1}, \dot{\zeta^\star}
            ]
  - where \zeta \in R^n, n being the dimension in cartesian coordinates of the 
  - note that our data is in 2D for now, i.e. x and y axes in Cartesian space
  
  In other words, data is thus shaped:
                  
  data = 
                  point p1                               point p2                          point p3                  target joint space angles
         [x1,  x2,  x3,  x4,  x5,  x6,  x7 | x1,  x2,  x3,  x4,  x5,  x6,  x7 | x1,  x2,  x3,  x4,  x5,  x6,  x7 | x1,  x2,  x3,  x4,  x5,  x6,  x7 ]
         [y1,  y2,  y3,  y4,  y5,  y6,  y7 | y1,  y2,  y3,  y4,  y5,  y6,  y7 | y1,  y2,  y3,  y4,  y5,  y6,  y7 | y1,  y2,  y3,  y4,  y5,  y6,  y7 ]
         [x1d, x2d, x3d, x4d, x5d, x6d, x7d| x1d, x2d, x3d, x4d, x5d, x6d, x7d| x1d, x2d, x3d, x4d, x5d, x6d, x7d| x1d, x2d, x3d, x4d, x5d, x6d, x7d]
         [y1d, y2d, y3d, y4d, y5d, y6d, y7d| y1d, y2d, y3d, y4d, y5d, y6d, y7d| y1d, y2d, y3d, y4d, y5d, y6d, y7d| y1d, y2d, y3d, y4d, y5d, y6d, y7d]

In [2]:
filename = '../scripts/{}.h5'.format('torobo_processed_data')
with h5py.File(filename, 'r+') as f:
    data = f['data/data'].value
print(data.shape)

# gmm = GMM()

(4, 21)


In [23]:
x0 = data[:, :14]
xT = data[:, 14:]

#### now compute the priors, mus and sigmas of the Gaussian Mixture Model

In [4]:
def guess_init_lyap(data, Vxf0, b_initRandom=False):
    """
    This function guesses the initial lyapunov function
    """
    # allocate spaces for incoming arrays
    Vxf0['Mu']  =  np.zeros(( Vxf0['d'], Vxf0['L']+1 )) # will be 2x2
    Vxf0['P']   =  np.zeros(( Vxf0['d'], Vxf0['d'], Vxf0['L']+1)) # wil be 2x2x3

    if b_initRandom:
        lengthScale = np.sqrt(np.var(data[:Vxf0['d'],:].T, axis=0))
        lengthScale = np.ravel(lengthScale)
        '''
         If `rowvar` is True (default), then each row represents a
        variable, with observations in the columns. Otherwise, the relationship
        is transposed: each column represents a variable, while the rows
        contain observations.
        '''
        #tempcov = np.cov(np.var(data[:Vxf0['d'],:], axis=0), rowvar=False)
        lengthScaleMatrix = LA.sqrtm(np.cov(np.var(data[:Vxf0['d'],:], axis=0), rowvar=False))
        Vxf0['Priors'] = np.random.rand(Vxf0['L']+1,1)

        for l in range(Vxf0['L']+1):
            tempMat = np.random.randn(Vxf0['d'], Vxf0['d'])
            Vxf0['Mu'][:,l] = np.multiply(np.random.randn(Vxf0['d'],1), lengthScale)
            Vxf0['P'][:,:,l] = lengthScaleMatrix.dot((tempMat * tempMat.T)).dot(lengthScaleMatrix)
    else:
        Vxf0['Priors'] = np.ones((Vxf0['L']+1, 1))
        Vxf0['Priors'] = Vxf0['Priors']/np.sum(Vxf0['Priors'])
        Vxf0['Mu'] = np.zeros((Vxf0['d'], Vxf0['L']+1))

        Vxf0['P']   =  np.zeros(( Vxf0[ 'd'], Vxf0['d'], Vxf0['L']+1)) # wil be 2x2x3
        for l in range(Vxf0['L']+1):
            Vxf0['P'][:,:,l] = np.eye((Vxf0['d']))

    Vxf0.update(Vxf0)

    return Vxf0

In [5]:
Vxf0 = {
    'L': None,
    'd': None,
    'w': 1e-4, #A positive scalar weight regulating the priority between the two objectives of the opitmization. Please refer to the page 7 of the paper for further information.
    'Mu': np.array(()),
    'P': np.array(()),
}

options = {
    'tol_mat_bias': 1e-1,
    'display': 1,
    'tol_stopping': 1e-10,
    'max_iter': 500,
    'optimizePriors': True,
    'upperBoundEigenValue': True,
}


hyperparams = {
    'use_cvxopt': True, #whether to use cvxopt package, fmincon or otherwise
}

options = {
    'tol_mat_bias': 1e-1,
    'display': 1,
    'tol_stopping': 1e-10,
    'max_iter':500,
    'optimizePriors': True,
    'upperBoundEigenValue': True
}

""" This file defines a Gaussian mixture model class. """

def logsum(vec, axis=0, keepdims=True):
    #TODO: Add a docstring.
    maxv = np.max(vec, axis=axis, keepdims=keepdims)
    maxv[maxv == -float('inf')] = 0
    return np.log(np.sum(np.exp(vec-maxv), axis=axis, keepdims=keepdims)) + maxv

def check_sigma(A):
    """
        checks if the sigma matrix is symmetric
        positive definite before inverting via cholesky decomposition
    """
    eigval = np.linalg.eigh(A)[0]
    if np.array_equal(A, A.T) and np.all(eigval>0):
        # LOGGER.debug("sigma is pos. def. Computing cholesky factorization")
        return A
    else:
        # find lowest eigen value
        eta = 1e-6  # regularizer for matrix multiplier
        low = np.amin(np.sort(eigval))
        Anew = low * A + eta * np.eye(A.shape[0])
        return Anew

class GMM(object):
    """ Gaussian Mixture Model. """
    def __init__(self, init_sequential=False, eigreg=False, warmstart=True):
        self.init_sequential = init_sequential
        self.eigreg = eigreg
        self.warmstart = warmstart
        self.sigma = None

    def inference(self, pts):
        """
        Evaluate dynamics prior.
        Args:
            pts: A N x D array of points.
        """
        # Compute posterior cluster weights.
        logwts = self.clusterwts(pts)

        # Compute posterior mean and covariance.
        mu0, Phi = self.moments(logwts)

        # Set hyperparameters.
        m = self.N
        n0 = m - 2 - mu0.shape[0]

        # Normalize.
        m = float(m) / self.N
        n0 = float(n0) / self.N
        return mu0, Phi, m, n0

    def clusterwts(self, data):
        """
        Compute cluster weights for specified points under GMM.
        Args:
            data: An N x D array of points
        Returns:
            A K x 1 array of average cluster log probabilities.
        """
        # Compute probability of each point under each cluster.
        logobs = self.estep(data)

        # Renormalize to get cluster weights.
        logwts = logobs - logsum(logobs, axis=1)

        # Average the cluster probabilities.
        logwts = logsum(logwts, axis=0) - np.log(data.shape[0])
        return logwts.T

    def estep(self, data):
        """
        Compute log observation probabilities under GMM.
        Args:
            data: A N x D array of points.
        Returns:
            logobs: A N x K array of log probabilities (for each point
                on each cluster).
        """
        # Constants.
        N, D = data.shape
        K = self.sigma.shape[0]

        logobs = -0.5*np.ones((N, K))*D*np.log(2*np.pi)
        for i in range(K):
            mu, sigma = self.mu[i], self.sigma[i]
            sigma = sigma
            L = scipy.linalg.cholesky(sigma, lower=True)
            logobs[:, i] -= np.sum(np.log(np.diag(L)))
            diff = (data - mu).T
            soln = scipy.linalg.solve_triangular(L, diff, lower=True)
            logobs[:, i] -= 0.5*np.sum(soln**2, axis=0)

        logobs += self.logmass.T
        return logobs

    def moments(self, logwts):
        """
            Compute the moments of the cluster mixture with logwts.
            Args:
                logwts: A K x 1 array of log cluster probabilities.
            Returns:
                mu: A (D,) mean vector.
                sigma: A D x D covariance matrix.
        """
        # Exponentiate.
        wts = np.exp(logwts)

        # Compute overall mean.
        mu = np.sum(self.mu * wts, axis=0)

        # Compute overall covariance.
        diff = self.mu - np.expand_dims(mu, axis=0)
        diff_expand = np.expand_dims(self.mu, axis=1) * \
                np.expand_dims(diff, axis=2)
        wts_expand = np.expand_dims(wts, axis=2)
        sigma = np.sum((self.sigma + diff_expand) * wts_expand, axis=0)
        return mu, sigma

    def update(self, data, K, max_iterations=100):
        """
        Run EM to update clusters.
        Args:
            data: An N x D data matrix, where N = number of data points.
            K: Number of clusters to use.
        """
        # Constants.
        N  = data.shape[0]
        Do = data.shape[1]

        LOGGER.debug('Fitting GMM with %d clusters on %d points.', K, N)

        if (not self.warmstart or self.sigma is None or K != self.sigma.shape[0]):
            # Initialization.
            LOGGER.debug('Initializing GMM.')
            self.sigma = np.zeros((K, Do, Do))
            self.mu = np.zeros((K, Do))
            self.logmass = np.log(1.0 / K) * np.ones((K, 1))
            self.mass = (1.0 / K) * np.ones((K, 1))
            self.N = data.shape[0]
            N = self.N

            # Set initial cluster indices.
            if not self.init_sequential:
                cidx = np.random.randint(0, K, size=(1, N))
            else:
                raise NotImplementedError()

            # Initialize.
            for i in range(K):
                cluster_idx = (cidx == i)[0]
                mu = np.mean(data[cluster_idx, :], axis=0)
                diff = (data[cluster_idx, :] - mu).T
                sigma = (1.0 / K) * (diff.dot(diff.T))
                self.mu[i, :] = mu
                self.sigma[i, :, :] = sigma + np.eye(Do) * 2e-6

        prevll = -float('inf')
        for itr in range(max_iterations):
            # E-step: compute cluster probabilities.
            logobs = self.estep(data)

            # Compute log-likelihood.
            ll = np.sum(logsum(logobs, axis=1))
            LOGGER.debug('GMM itr %d/%d. Log likelihood: %f',
                         itr, max_iterations, ll)
            if ll < prevll:
                # TODO: Why does log-likelihood decrease sometimes?
                LOGGER.debug('Log-likelihood decreased! Ending on itr=%d/%d',
                             itr, max_iterations)
                break
            if np.abs(ll-prevll) < 1e-5*prevll:
                LOGGER.debug('GMM converged on itr=%d/%d',
                             itr, max_iterations)
                break
            prevll = ll

            # Renormalize to get cluster weights.
            logw = logobs - logsum(logobs, axis=1)
            assert logw.shape == (N, K)

            # Renormalize again to get weights for refitting clusters.
            logwn = logw - logsum(logw, axis=0)
            assert logwn.shape == (N, K)
            w = np.exp(logwn)

            # M-step: update clusters.
            # Fit cluster mass.
            self.logmass = logsum(logw, axis=0).T
            self.logmass = self.logmass - logsum(self.logmass, axis=0)
            assert self.logmass.shape == (K, 1)
            self.mass = np.exp(self.logmass)

            # Reboot small clusters.
            w[:, (self.mass < (1.0 / K) * 1e-4)[:, 0]] = 1.0 / N
            # Fit cluster means.
            w_expand = np.expand_dims(w, axis=2)
            data_expand = np.expand_dims(data, axis=1)
            self.mu = np.sum(w_expand * data_expand, axis=0)
            # Fit covariances.
            wdata = data_expand * np.sqrt(w_expand)
            assert wdata.shape == (N, K, Do)
            for i in range(K):
                # Compute weighted outer product.
                XX = wdata[:, i, :].T.dot(wdata[:, i, :])
                mu = self.mu[i, :]
                self.sigma[i, :, :] = XX - np.outer(mu, mu)

                if self.eigreg:  # Use eigenvalue regularization.
                    raise NotImplementedError()
                else:  # Use quick and dirty regularization.
                    sigma = self.sigma[i, :, :]
                    self.sigma[i, :, :] = 0.5 * (sigma + sigma.T) + \
                            1e-6 * np.eye(Do)
                    
def get_pdf(data, mu, sigma):
    nbVar, nbdata = data.shape

    data = data.T - np.tile(mu.T, [nbdata,1])
    prob = np.sum((data/sigma)*data, axis=1);
    prob = np.exp(-0.5*prob) / np.sqrt((2*np.pi)**nbVar *
                                       (np.abs(np.linalg.det(sigma))+
                                        np.finifo(np.float64).min))
    return prob

def regress_gauss_mix(Priors, Mu, Sigma, x, inp, out, nargout=3):
    nbData = x.shape[1]
    nbVar = Mu.shape[0]
    nbStates = Sigma.shape[2]

    Pxi = np.zeros_like(Priors)
    for i in range(nbStates):
        Pxi[:,i] = Priors[i] * get_pdf(x, Mu[inp,i], Sigma[inp,inp,i])

    beta = Pxi / np.tile(np.sum(Pxi,axis=1) +
                         np.finfo(np.float32).min, [1,nbStates])
    #########################################################################
    for j in range(nbStates):
        y_tmp[:,:,j] = np.tile(Mu[out,j],[1,nbData]) \
                     + Sigma[out,inp,j]/(Sigma[inp,inp,j]).dot(x-np.tile(Mu[inp,j],[1,nbData]))

    beta_tmp = beta.reshape(1, beta.shape)
    y_tmp2 = np.tile(beta_tmp,[matlength(out), 1, 1]) * y_tmp
    y = np.sum(y_tmp2,axis=2)
    ## Compute expected covariance matrices Sigma_y, given input x
    #########################################################################
    if nargout > 1:
        for j in range(nbStates):
            Sigma_y_tmp[:,:,0,j] = Sigma[out,out,j] \
                                   - (Sigma[out,inp,j]/(Sigma[inp,inp,j])  \
                                   * Sigma[inp,out,j])

        beta_tmp = beta.reshape(1, 1, beta.shape)
        Sigma_y_tmp2 = np.tile(beta_tmp * beta_tmp, \
                               [matlength(out), matlength(out), 1, 1]) * \
                                np.tile(Sigma_y_tmp,[1, 1, nbData, 1])
        Sigma_y = np.sum(Sigma_y_tmp2, axis=3)

    return y, Sigma_y, beta

def obj(p, x, xd, d, L, w, options):
    Vxf         = gauss_params_to_lyapunov(p,d,L,options)
    _, Vx       = compute_lyapunov(x, None, Vxf)
    Vdot        = np.sum(Vx*xd, axis=0)  #derivative of J w.r.t. xd
    norm_Vx     = np.sqrt(np.sum(Vx * Vx, axis=0))
    norm_xd     = np.sqrt(np.sum(xd * xd, axis=0))
    J           = np.divide(Vdot, np.multiply(norm_Vx, norm_xd))#.squeeze()

    # projections onto positive orthant
    J[np.where(norm_xd==0)] = 0
    J[np.where(norm_Vx==0)] = 0
    J[np.where(Vdot>0)]     = J[np.where(Vdot>0)]**2      # solves psi(t,n)**2
    J[np.where(Vdot<0)]     = -w*J[np.where(Vdot<0)]**2   # # J should be (1, 750)
    J                       = np.sum(J)
    dJ                      = None
    
    return J#, dJ

def optimize(obj_handle, p0):
    opt = minimize(
        obj_handle,
        x0=p0,
        method='L-BFGS-B',
        jac=False,
        bounds=[(0.0, None) for _ in range(len(p0))], # no negative p values
        #bounds = Bounds(ctr_handle(p0), keep_feasible=True), # will produce c, ceq as lb and ub
        options={'ftol': 1e-4, 'disp': True}
        )
    return opt
        
def stack_gmm_params(Vxf, options):
    # transforming optimization parameters into a column vector
    d = Vxf['d']
    if Vxf['L'] > 0:
        if options['optimizePriors']:
            p0 = np.vstack((
                           np.expand_dims(np.ravel(Vxf['Priors']), axis=1), # will be a x 1
                           np.expand_dims(Vxf['Mu'][:,1:], axis=1).reshape(Vxf['L']*d,1)
                        ))
        else:
            p0 = Vxf['Mu'][:,2:].reshape(Vxf['L']*d, 1) #print(p0) # p0 will be 4x1
    else:
        p0 = []

    for k in range(Vxf['L']):
        p0 = np.vstack((
                      p0,
                      Vxf['P'][:,:,k+1].reshape(d**2,1)
                    ))
    return p0

def parameters_2_gmm(popt, d, L, options):
    # transforming the column of parameters into Priors, Mu, and P
    Vxf = gauss_params_to_lyapunov(popt, d, L, options)

    return Vxf

def gauss_params_to_lyapunov(p,d,L,options):
    # transforming the column of parameters into Priors, Mu, and P
    P = np.zeros((d,d,L+1))
    optimizePriors = options['optimizePriors']
    if L == 0:
        Priors = 1
        Mu = np.zeros((d,1))
        i_c = 1
    else:
        if options['optimizePriors']:
            Priors = p[:L+1]
            i_c = L+1
        else:
            Priors = np.ones((L+1,1))
            i_c = 0

        Priors = np.divide(Priors, np.sum(Priors))
        Mu = np.hstack((np.zeros((d,1)), p[[i_c+ x for x in range(d*L)]].reshape(d,L)))
        i_c = i_c+d*L+1

    for k in range(L):
        P[:,:,k+1] = p[range(i_c+k*(d**2)-1,i_c+(k+1)*(d**2)-1)].reshape(d,d)

    Vxf           = dict()
    Vxf['Priors'] = Priors
    Vxf['Mu']     = Mu
    Vxf['P']      = P
    Vxf['SOS']    = 0
    
    return Vxf

def matVecNorm(x):
    return np.sqrt(np.sum(x**2, axis=0))

def matlength(x):
  # find the max of a numpy matrix dims
  return np.max(x.shape)

def ctr_eigenvalue(p,d,L,options):
    Vxf = dict()
    if L == -1: # SOS
        Vxf['d'] = d
        Vxf['n'] = int(np.sqrt(matlength(p)/d**2))
        Vxf['P'] = p.reshape(Vxf['n']*d,Vxf['n']*d)
        Vxf['SOS'] = 1
        c  = np.zeros((Vxf['n']*d))
        ceq = []
    else:
        Vxf = gauss_params_to_lyapunov(p,d,L,options)
        if L > 0:
            c  = np.zeros([(L+1)*d+(L+1)*options['optimizePriors']])  #+options.variableSwitch
            if options['upperBoundEigenValue']:
                ceq = np.zeros((L+1))
            else:
                ceq = [] 
        else:
            c  = np.zeros((d))
            ceq = Vxf['P'].T.ravel().dot(Vxf['P'].ravel()) - 2

    dc = [] 
    dceq = [] 

    if L == -1:  # SOS
        c = -LA.eigvals(Vxf['P'] + Vxf['P'].T - np.eye(Vxf['n']*d)*options['tol_mat_bias'])
    else:
        for k in range(L):
            lambder = LA.eigvals(Vxf['P'][:,:,k+1] + (Vxf['P'][:,:,k+1]).T)/2.0
            c[k*d:(k+1)*d] = -lambder.real + options['tol_mat_bias']
            if options['upperBoundEigenValue']:
                ceq[k+1] = 1.0 - np.sum(lambder.real) 

    if L > 0 and options['optimizePriors']:
        c[(L+1)*d:(L+1)*d+L+1] = -Vxf['Priors'].squeeze()

    return c, ceq 

def optimize_lyapunov(Vxf0, Data, options):
    d = int(Data.shape[0]/2)  
    x = Data[:d,:]     
    xd = Data[d:2*d,:]  
    Vxf0['SOS'] = False
    
    # Transform the Lyapunov model to a vector of optimization parameters
    if Vxf0['SOS']:
        p0 = npr.randn(d*Vxf0['n'], d*Vxf0['n']);
        p0 = p0.dot(p0.T)
        p0 = np.ravel(p0)
        Vxf0['L'] = -1; # to distinguish sos from other methods
    else:
        for l in range(Vxf0['L']):
            try:
                Vxf0['P'][:,:,l+1] = scipy.linalg.solve(Vxf0['P'][:,:,l+1], np.eye(d))
                #print('Vxf0[:,:,l+1]: ', Vxf0['P'][:,:,l+1])
            except LA.LinAlgError as e:
                LOGGER.debug('LinAlgError: %s', e)

        # in order to set the first component to be the closest Gaussian to origin
        to_sort = matVecNorm(Vxf0['Mu'])
        idx = np.argsort(to_sort, kind='mergesort')
        Vxf0['Mu'] = Vxf0['Mu'][:,idx]
        Vxf0['P']  = Vxf0['P'][:,:,idx]
        p0 = stack_gmm_params(Vxf0,options)

    obj_handle = lambda p: obj(p, x, xd, d, Vxf0['L'], Vxf0['w'], options)
    ctr_handle = lambda p: ctr_eigenvalue(p, d, Vxf0['L'], options)
    
    optim_res = optimize(obj_handle, p0) 
    popt, J = optim_res.x, optim_res.fun

    if Vxf0['SOS']:
        Vxf['d']    = d
        Vxf['n']    = Vxf0['n']
        Vxf['P']    = popt.reshape(Vxf['n']*d,Vxf['n']*d)
        Vxf['SOS']  = 1
        Vxf['p0']   = compute_Energy(zeros(d,1),[],Vxf)
        #check_constraints(popt,ctr_handle,d,0,options)
    else:
        # transforming back the optimization parameters into the GMM model
        Vxf             = parameters_2_gmm(popt,d,Vxf0['L'],options)
        Vxf['Mu'][:,0]  = 0
        Vxf['L']        = Vxf0['L']
        Vxf['d']        = Vxf0['d']
        Vxf['w']        = Vxf0['w']
        #check_constraints(popt,ctr_handle,d,Vxf['L'],options)

    sumDet = 0
    for l in range(Vxf['L']+1):
        sumDet += np.linalg.det(Vxf['P'][:,:,l])

    Vxf['P'][:,:,0] = Vxf['P'][:,:,0]/sumDet
    Vxf['P'][:,:,1:] = Vxf['P'][:,:,1:]/np.sqrt(sumDet)

    return Vxf, J

def compute_lyapunov(X,Xd,Vxf, nargout=2):
    d = X.shape[0]
    nDemo = 1 
    if nDemo>1:
        X = X.reshape(d,-1)
        Xd = Xd.reshape(d,-1)

    if Vxf['SOS']:
        V, dV = sos_lyapunov(X, Vxf['P'], Vxf['d'], Vxf['n'])
        if 'p0' in Vxf:
            V -= Vxf['p0']
    else:
        V, dV = gauss_regress_to_lyapunov(X, Vxf['Priors'], Vxf['Mu'], Vxf['P'])

    if nargout > 1:
        if not Xd:
            Vdot = dV
        else:
            Vdot = np.sum(Xd*dV, axis=0)
    if nDemo>1:
        V = V.reshape(-1, nDemo).T
        if nargout > 1:
            Vdot = Vdot.reshape(-1, nDemo).T

    return V, Vdot


def gauss_regress_to_lyapunov(x, Priors, Mu, P):
    # print('x.shape: ', x.shape)
    nbData = x.shape[1]
    d = x.shape[0]
    L = P.shape[2]-1;

    # Compute the influence of each GMM component, given input x
    for k in range(L):
        P_cur               = P[:,:,k+1]
        if k                == 0:
            V_k             = np.sum(x * (P_cur.dot(x)), axis=0)
            V               = Priors[k+1]*(V_k)
            Vx              = Priors[k+1]*((P_cur+P_cur.T).dot(x))
        else:
            x_tmp           = x - np.tile(Mu[:,k+1], [nbData, 1]).T
            V_k             = np.sum(P_cur.dot(x_tmp)*x, axis=0)
            V_k[V_k < 0]    = 0
            V              += Priors[k+1] * (V_k ** 2)
            temp            = (2 * Priors[k+1]) * (V_k)
            Vx              = Vx + np.tile(temp, [d,1])*(P_cur.dot(x_tmp) + P_cur.T.dot(x))
    
    return V, Vx


In [6]:
def dsStabilizer(X, fn_handle, Vxf, rho0, kappa0):
    d = Vxf['d']
    if X.shape[0] == 2*d:
        Xd     = X[d:2*d,:]
        X      = X[:d,:]
    else:
        if (len(getfullargspec(fn_handle).args) == 1):
            Xd, _, _ = fn_handle(X)
        elif (len(getfullargspec(fn_handle).args) == 2):
            t  = X[d,:]
            X  = X[d:]
            Xd, _, _ = fn_handle(t,X)
        else:
            logger.CRITICAL('Unknown function handle!')

    V,Vx    = compute_lyapunov(X,[],Vxf)
    norm_Vx = np.sum(V ** 2, axis=0)
    norm_x  = np.sum(X ** 2,axis=0)
    
    Vdot    = np.sum(Vx * Xd,axis=0)
    rho     = rho0 * (1-np.exp(-kappa0*norm_x)) * np.sqrt(norm_Vx)
    ind     = (Vdot + rho) >= 0
    #ind     = np.where((Vdot + rho) >= 0)
    u       = Xd * 0
    
    print(np.sum(ind))
    if np.sum(ind)>0:
        lambder   = (Vdot[ind] + rho[ind]) / norm_Vx[ind]
        u[:,ind]  = -np.tile(lambder,[d,1]) * Vx[:,ind]
        Xd[:,ind] = Xd[:,ind] + u[:,ind]

#     if args:
#         dt = args[0]
#         xn = X + np.dot(Xd, dt)
#         Vn = compute_lyapunov(xn,[],Vxf)
#         ind = (Vn >= V)
#         i = 0

#         while(np.any(ind) and i < 10):
#             alpha = np.divide(V[ind], Vn[ind])
#             Xd[:,ind] = np.tile(alpha,[d,1]) * Xd[:,ind] - \
#                         np.tile(alpha * np.sum(Xd[:,ind] * \
#                         Vx[:,ind], axis=0)/norm_Vx[ind],[d,1])*Vx[:,ind]
#             xn = X + np.dot(Xd,dt)
#             Vn = compute_lyapunov(xn,np.array(()),Vxf)
#             ind = (Vn >= V)
#             i = i + 1

    return Xd, u

In [7]:
Vxf0['L'] = 2   # number of asymmetric quadratic components for L >= 0
Vxf0['d'] = int(data.shape[0]/2)
Vxf0.update(Vxf0)
# A set of options that will be passed to the solver
options = {
    'tol_mat_bias': 1e-1,
    'display': 1,
    'tol_stopping': 1e-10,
    'max_iter':500,
    'optimizePriors': True,
    'upperBoundEigenValue': True
}

Vxf0 = guess_init_lyap(data, Vxf0, b_initRandom=False)
# for k, v in Vxf0.items():
#     print(k, v)
# print('\n')
Vxf, J = optimize_lyapunov(Vxf0, data, options)
# for k, v in Vxf.items():
#     print(k, v)
# print('J: ', J)`
# print('\n')
gmm.update(data.T, K=6, max_iterations=100)
mu, sigma, priors = gmm.mu, gmm.sigma, gmm.logmass

print(mu.shape, sigma.shape, priors.shape)
inp = range(0, Vxf['d'])
out = range(Vxf['d'], 2* Vxf['d'])
gmr_handle = lambda x: regress_gauss_mix(priors, mu, sigma, x, inp, out)
rho0, kappa0 = 1.0, 1.0
Xd, u = dsStabilizer(data, gmr_handle, Vxf, rho0, kappa0)
print(Xd.shape, u.shape)

/home/olalekan/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:292: RuntimeWarning: invalid value encountered in true_divide
/home/olalekan/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:473: RuntimeWarning: invalid value encountered in sqrt


NameError: name 'gmm' is not defined

In [8]:
def ctr_eigenvalue(p,d,L,options):
    Vxf = dict()
    if L == -1: # SOS
        Vxf['d'] = d
        Vxf['n'] = int(np.sqrt(matlength(p)/d**2))
        Vxf['P'] = p.reshape(Vxf['n']*d,Vxf['n']*d)
        Vxf['SOS'] = 1
        c  = np.zeros(( Vxf['n']*d, 1 ))
        ceq = []
    else:
        Vxf = gauss_params_to_lyapunov(p,d,L,options)
        if L > 0:
            c  = np.zeros(((L+1)*d+(L+1)*options['optimizePriors'],1)) 
            if options['upperBoundEigenValue']:
                ceq = np.zeros((L+1,1))
            else:
                ceq = []
        else:
            c  = np.zeros((d,1))
            ceq = (np.ravel(Vxf['P']).T).dot(np.ravel(Vxf['P'])) -2

    dc, dceq = [], []

    if L == -1:  # SOS
        c = -np.linalg.eigvals(Vxf['P'] + Vxf['P'].T - np.eye(Vxf['n']*d)*options['tol_mat_bias'])
    else:
        for k in range(L):
            lambder = sp.linalg.eigvals(Vxf['P'][:,:,k+1] + (Vxf['P'][:,:,k+1]).T).real/2.0
            lambder = np.expand_dims(lambder, axis=1)
            c[k*d:(k+1)*d] = -lambder + options['tol_mat_bias']
            if options['upperBoundEigenValue']:
                ceq[k+1] = 1.0 - np.sum(lambder.real) # + Vxf.P(:,:,k+1)'

        if L > 0 and options['optimizePriors']:
            c[(L+1)*d:(L+1)*d+L+1] = -Vxf['Priors']

    return c, ceq, dc, dceq

In [9]:
import numpy as np
ind = np.array([ True,  True , True , True , True , True  ,True  ,True ,False , True ,False,
 False  ,True,  True , True,  True  ,True , True,  True,  True])
print(ind.shape)
print(np.sum(ind))

(20,)
17


In [10]:
import sys
sys.path.append('/Users/Lekan/catkin_ws/src/torobo/tampy')
sys.path.append('/Users/Lekan/catkin_ws/src/dp_planning')
from tampy.tampy import Tampy
from tampy.tampy import ORDER_SERVO_ON, ORDER_SERVO_OFF, ORDER_RUN_MODE, CTRL_MODE_CURRENT

class ToroboEnvironment(object):
    def __init__(self, home_pos):
        """
         in update(self, currents) currents are inindividual currents to each joint
         According to Ryo-san, the max current is 0.5A and you should start with 0.05A or sth and gradually increase
        """
        self.tampy = Tampy()
        self.home_pos = home_pos
        self.control_freq = 30.0
        self.latest_control = time.time()

    def set_position(self, positions):
        self.tampy.move_to(positions)

    def update(self, currents):
        self.tampy.send_currents(currents)
        time.sleep(max(
            self.latest_control + 1.0 / self.control_freq - time.time(),
            0
        ))
        self.latest_control = time.time()
        return self.get_state()

    def get_state(self):
        rx = self.tampy.get_latest_rx()
        positions = [j.position for j in rx.joints]
        velocities = [j.velocity for j in rx.joints]
        return positions, velocities

    def __enter__(self):
        self.set_position(self.home_pos)
        self.tampy.send(ORDER_RUN_MODE, value1=CTRL_MODE_CURRENT)
        self.tampy.send(ORDER_SERVO_ON)
        self.latest_control = time.time()
        return self

    def __exit__(self, type, value, tb):
        self.tampy.send_currents([0] * 7)
        # TODO: why do we need multiple calls to kill?
        for _ in range(3):
            self.tampy.send(ORDER_SERVO_OFF)


ImportError: No module named 'tampy'

In [11]:
def robot_exec(x0, xT, ds_stab, options):
    d = x0.shape[0]
    nbSPoint = x0.shape[1]
    
    if not xT:
        xT = np.zeros((d, 1))
    
    if d != xT.shape[0]:
        logger.critical('Error: length x0 should be length xT')
        x, xd, t = [], [], []
        return
    
    #################################################
    #                   starting values             #
    #################################################
    x  = np.zeros((d, 0, nbSPoint))
    xd = np.zeros((d, 0, nbSPoint))
    for i in range(nbSPoint):
        x[:, 0, i] = x0[:,i]
    xd = np.zeros(x.shape)
    
    if xT.shape == x0.shape:
        XT = xT
    else:
        XT = np.tile(xT, [1, nbSPoint])
    
    t = 0
    # robot run
    i = 0
#     while True:
#         xd[:,i,:] = np.tile(ds_stab(np.squeeze(x[:,i,:])-XT), [d, 1, nbSPoint])
        
        
#         ### integration ###
#         x[:,i+1,:] = x[:,i,:] + xd[:,i,:] * options['dt']
#         t[i+1] = t[i]+options['dt']
        
#         i += 1
        
#         # check convergence
#         if i > 3 and (np.all(np.all(np.all(abs(xd[:,-1-3:-1,:])))))
    while np.linalg.norm():
        
     

SyntaxError: unexpected EOF while parsing (<ipython-input-11-a7db2a28c25b>, line 44)

In [24]:
filepath = join(expanduser('~'), 'Documents', 'LyapunovLearner', 'ToroboTakahashi', 'data')
name = 'state_joint.npy'
filename = join(filepath, name)
print(filename)

/home/olalekan/Documents/LyapunovLearner/ToroboTakahashi/data/state_joint.npy


In [25]:
os.listdir(filepath)

['state_joint.npy.bak',
 'state_joint_pos_only.csv',
 'state_joint_pos_only.npy',
 'state_joint.npy']

In [26]:
data = np.load(filename)
print(data.shape, data.dtype)

(10001, 25) float64


np.save(filepath + '/state_joint_pos_only.npy', data[:, 1:8])
np.savetxt(filepath + '/state_joint_pos_only.csv', data[:,1:8], delimiter=" ", newline="\n")

pos_data = np.loadtxt(filepath + '/state_joint_pos_only.csv')
pos_data.shape

new_data = np.ravel(pos_data)
rec_data = new_data.reshape(10001, 7)
print(rec_data)

### prepro new csv data saved from c++

In [4]:
filepath = join(expanduser('~'), 'Documents', 'LyapunovLearner', 'scripts', 'data')
name = 'cart_pos.csv'
filename = join(filepath, name)
print(os.path.isfile(filename))

True


In [5]:
with open(filename, 'r+') as foo:
    #print(filename)
    data = foo.readlines()


In [6]:
# gather data into right shape
proper_data = []

for i in range(len(data)):
    temp = data[i].rsplit(sep=',')
    temp[-1] = temp[-1].rsplit(sep='\n')[0]    
    temp = [float(x) for x in temp]
    proper_data.append(temp)
    
proper_data = np.array(proper_data)
print(proper_data.shape)

(10000, 6)


In [7]:
# gather 2d data
data2d = np.hstack([proper_data[:, :2], proper_data[:, 3:5]]).T
print(data2d.shape)

(4, 10000)


In [11]:
proper_data.shape

(10000, 6)